In [ ]:
#모듈
import math
import pandas as pd
import numpy as np

try:
    import cufflinks as cf
except:
    !pip install cufflinks
    import cufflinks as cf

cf.go_offline()

from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 다루기
pd.set_option("Display.max_columns",200)
import re
import time

# 데이터 시각화
import seaborn as sns

# 사용할 머신러닝 모델들 - RandomForest, AdaBoost, GradientBoost, ExtraTrees
import sklearn
from sklearn.metrics import log_loss
from sklearn.svm import SVC

In [ ]:
#명목형 변수 더미화
def dummify_category_cols(df,cols):
    dummies = []
    new_df = df.copy()

    for idx in range(len(cols)):
        new_df = new_df.join(pd.get_dummies(total[cols[idx]], prefix = cols[idx]))
        new_df = new_df.drop(cols[idx], axis =1)
    return new_df

In [ ]:
#숫자형 변수 범주화
def dummify_number_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: int(x)//num * num)
    return new_df


In [ ]:
#명목형 변수의 연속형 변환 - 변수의 차원을 낮추기 위하여
def num_category(df, col, chg) :
    new_df = df.copy()
    set = {}
    
    for idx in range(len(chg)) :
        set[chg[idx]] = idx
        
    new_df[col] = new_df[col].map(set)
    
    return new_df        

In [ ]:
#숫자형 변수 일정 이상은 묶어버리기
def number_group_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: num if x> num else x )
    return new_df


In [ ]:
#RMS 구하기
def RMSE(y_true,y_pred):
    
    squaredErrors = (y_pred - y_true)**2
    MSE = squaredErrors.mean()
    
    return math.sqrt(MSE)

In [ ]:
#본격적으로 시작

In [ ]:
training = pd.read_csv("house_price.csv")
test = pd.read_csv("test.csv")

#training = training[np.isfinite(training["GarageYrBlt"])]

In [ ]:
total = pd.concat([training, test]).reset_index()
del total["index"]

In [ ]:
total.head(2)

In [ ]:
total.columns

In [ ]:
#선택된 변수
category_selected = [
            "Neighborhood", "OverallQual", 
            "MasVnrType", "ExterQual",
            "BsmtQual", "BsmtFinType1",
            "FullBath", "KitchenQual",
            "TotRmsAbvGrd", "Fireplaces", "FireplaceQu",
            "GarageCars"]

extra_selected = [ "GarageCars","OverallQual","OverallCond","ExterQual","BsmtQual","BsmtExposure", "BsmtFinType1","KitchenQual","FireplaceQu"]

number_selected = ["GarageYrBlt","YearBuilt", "YearRemodAdd", "BsmtFinSF1", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "GrLivArea", 'GarageArea']

category_selected = [x for x in category_selected if x not in extra_selected]

total_selected = category_selected + number_selected + extra_selected

selected_with_SalePrice = total_selected +["SalePrice"]

In [ ]:
new_total = total[selected_with_SalePrice].copy()
new_total.head(2)

In [ ]:
#임시 전처리

In [ ]:
#전처리1 - 연속형데이터 중 null값이 있는 행 모두 버려주기
new_total[number_selected].isnull().sum()

In [ ]:
new_total["YearBuilt"] = new_total["YearBuilt"].fillna(new_total.BsmtFinSF1.mean())

In [ ]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].fillna(new_total.BsmtFinSF1.mean())

In [ ]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].fillna(new_total.BsmtUnfSF.mean())

In [ ]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].fillna(new_total.TotalBsmtSF.mean())

In [ ]:
new_total["GarageArea"] = new_total["GarageArea"].fillna(new_total.GarageArea.mean())

In [ ]:
new_total["GarageYrBlt"] = new_total["GarageYrBlt"].fillna(new_total.GarageYrBlt.mean())

In [ ]:
#new_total["Electrical"] = new_total["Electrical"].fillna("SBrkr")

In [ ]:
new_total = new_total.fillna("404")

In [ ]:
new_total.isnull().sum()

In [ ]:
new_total.head()

In [ ]:
#명목형 변수를 연속형으로 바꿔주는 것

In [ ]:
new_total = num_category(new_total, "ExterQual", ["Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
#new_total = num_category(new_total, "ExterCond", ["Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "BsmtQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
#new_total = num_category(new_total, "BsmtCond", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "BsmtExposure", ["404", "No", "Fa", "Mn", "Av", "Gd"])

In [ ]:
new_total = num_category(new_total, "BsmtFinType1", ["404", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"])

In [ ]:
#new_total = num_category(new_total, "HeatingQC", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "KitchenQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
new_total = num_category(new_total, "FireplaceQu", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [ ]:
#new_total = num_category(new_total, "Electrical", ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"])

In [ ]:
#new_total = num_category(new_total, "Functional", ["404", "Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"])
#new_total.Functional.unique()

In [ ]:
#변수끼리 합쳐주는 것

In [ ]:
#new_total["Functional"] = total["Functional"].map(lambda x : 1 if x==8 else 0)

In [ ]:
#new_total["LotConfig"] = new_total.LotConfig.map(lambda x : 1 if x == "CulDSac" else 0)

In [ ]:
new_total["FullBath"] = new_total.FullBath.map(lambda x : 1 if x <=2 else x-1)

In [ ]:
#반영 필요할듯!!!!
new_total["OverallCond"] = new_total.OverallCond.map(lambda x : 1 if x <=4 else 5 if x >= 8 else x-3)

In [ ]:
#반영 필요할드ㅛ!!!
new_total["OverallQual"] = new_total.OverallQual.map(lambda x : 1 if x <=4 else 5 if x>=8 else x-3)

In [ ]:
#new_total["BldgType"] = new_total.BldgType.map(lambda x : 1 if x =="1Fam" else 2)

In [ ]:
#반영 필요할듯!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#new_total = number_group_cols(new_total, ["TotRmsAbvGrd"], [13])
new_total["TotRmsAbvGrd"] = new_total.TotRmsAbvGrd.map(lambda x : 1 if x <=4 else 7 if x>=10 else x-3)

In [ ]:
#new_total = number_group_cols(new_total, ["HalfBath"], [1])

In [ ]:
#반영 필요할듯!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
new_total = number_group_cols(new_total, ["FullBath"], [2])

In [ ]:
#반영 필요할듯!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
new_total = number_group_cols(new_total, ["Fireplaces"], [2])

In [ ]:
#반영 필요할듯!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
new_total["GarageCars"] = new_total.GarageCars.map(lambda x : int(x))
new_total = number_group_cols(new_total, ["GarageCars"], [3])

In [ ]:
#임시 전처리

In [ ]:
#반영 필요할 듯!!!, 연도는 좀 묶어주는 것은 어떰? 적어도 멀리 떨어진 데이터
new_total = dummify_number_cols(new_total, ["GarageYrBlt"], [10])
new_total = dummify_number_cols(new_total, ["YearBuilt"], [10])
new_total["YearBuilt"] = new_total.YearBuilt.map(lambda x : 1900 if x <=1900 else 2000 if x>=2000 else x)
new_total = dummify_number_cols(new_total, ["YearBuilt"], [10])
new_total["YearRemodAdd"] = new_total.YearRemodAdd.map(lambda x : 1900 if x <=1900 else 2000 if x>=2000 else x)
new_total = dummify_number_cols(new_total, ["BsmtFinSF1"], [200])
new_total["BsmtFinSF1"] = new_total.BsmtFinSF1.map(lambda x : 1600 if x >=1600 else x)
new_total = dummify_number_cols(new_total, ["BsmtFinSF1"], [200])
new_total["BsmtFinSF1"] = new_total.BsmtFinSF1.map(lambda x : 1600 if x >=1600 else x)
new_total = dummify_number_cols(new_total, ["BsmtUnfSF"], [200])
new_total["BsmtUnfSF"] = new_total.BsmtUnfSF.map(lambda x : 1600 if x >=1600 else x)
new_total = dummify_number_cols(new_total, ["TotalBsmtSF"], [200])
new_total["TotalBsmtSF"] = new_total.TotalBsmtSF.map(lambda x : 1600 if x >=1600 else x)
new_total = dummify_number_cols(new_total, ["1stFlrSF"], [200])
new_total["1stFlrSF"] = new_total["1stFlrSF"].map(lambda x : 2000 if x >=1200 else x)
new_total = dummify_number_cols(new_total, ["2ndFlrSF"], [200])
new_total["2ndFlrSF"] = new_total["2ndFlrSF"].map(lambda x : 1400 if x >=1400 else x)
new_total = dummify_number_cols(new_total, ["GrLivArea"], [200])
new_total["GrLivArea"] = new_total.GrLivArea.map(lambda x : 2800 if x >=2800 else x)
new_total = dummify_number_cols(new_total, ["GarageArea"], [200])
new_total["GarageArea"] = new_total.GarageArea.map(lambda x : 800 if x >=800 else x)

In [ ]:
#반영 필요할드ㅛ!!!!!!!!!!!!
#1000개 이상 몰려 잇는 칼럼은 빼자...

In [ ]:
#확인

In [ ]:
new_total.head(2)

In [ ]:
#히트맵으로 확인
#training = training = new_total.iloc[:len(training)]

In [ ]:
#n= len(selected_with_SalePrice)
#colormap = plt.cm.viridis
#plt.figure(figsize=(n,n))
#plt.title('Pearson Correlation of Features', y=1.05, size=15)
#sns.heatmap(training.corr(),
#            linewidths=0.1,
#            vmax=1.0, 
#            square=True, 
#            cmap=colormap, 
#            linecolor='white', 
#            annot=True)

In [ ]:
new_total2 = dummify_category_cols(new_total,category_selected)

In [ ]:
new_total2.head(2)

In [ ]:
training = new_total2.iloc[:len(training)]
test = new_total2.iloc[len(training):]

In [ ]:
#여기서 selected 재설정

In [ ]:
training.head(2)

In [ ]:
training['is_train'] = np.random.uniform(0, 1, len(training)) <= .75

In [ ]:
training_set = training[training['is_train']==True]
test_set = training[training['is_train']==False]

In [ ]:
N = 4500
forest = RandomForestRegressor(n_estimators=N, n_jobs=2)

In [ ]:
feature = new_total2.columns.drop("SalePrice")
target = ["SalePrice"]

In [ ]:
forest.fit(training_set[feature], training_set[target])

In [ ]:
forest.predict(test_set[feature])

In [ ]:
forest.score(test_set[feature], test_set[target])

In [ ]:
y_pred=forest.predict(test_set[feature])

In [ ]:
y_true = test_set["SalePrice"]
rmse = RMSE(y_true,y_pred)
print(rmse)

In [ ]:
importance = pd.DataFrame(forest.feature_importances_, index = training_set[feature].columns)

In [ ]:
(
    importance.assign(header = lambda x: x.index.str.split("_").str[0])
                  .groupby("header")[0]
                  .sum()
                  .iplot(kind="bar")
)

In [ ]:
#최적화

In [ ]:
# 최적화할 파라미터 N
#best_score = 0
#for N in range(2000,6000,500):
#    tmp = RandomForestRegressor(n_estimators=N)
#    tmp.fit(training_set[feature], training_set[target])
#    score = tmp.score(test_set[feature],test_set[target])
#    
    # 만약 기존의 모델보다 더 예측률이 높다면, 최신화합니다
#    if score>best_score:
#        best_score = score
#        best_N = N
#        best_rf_model = tmp
        
#print("최적화 n_estimator 값 : {}".format(best_N))
#print("예측률 : {}".format(best_score))

In [ ]:
#best_score = 0
#for N in range(13,21):
#    tmp = RandomForestRegressor(n_estimators=4500, max_depth = N)
#    tmp.fit(training_set[feature], training_set[target])
#    score = tmp.score(test_set[feature],test_set[target])
#    
#    # 만약 기존의 모델보다 더 예측률이 높다면, 최신화합니다
#    if score>best_score:
#        best_score = score
#        best_N = N
#        best_rf_model = tmp
        
#print("최적화 max_depth 값 : {}".format(best_N))
#print("예측률 : {}".format(best_score))

In [ ]:
#제출

In [ ]:
forest.fit(training[feature], training[target])

In [ ]:
prediction =forest.predict(test[feature])

In [ ]:
submission = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction})

In [ ]:
submission.to_csv("Submission14.csv")